In [ ]:
import time
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from numpy import linalg
from decimal import *
from math import log
from prettytable import PrettyTable
from nltk.corpus import reuters
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [ ]:
# load in the corpus and go through each doc to get every word instance
vidx_corpus = pickle.load(open('reuters-clean-corpus-vidx.p', 'rb'))
corpus_vocab = pickle.load(open('reuters-clean-vocab.p', 'rb'))
corpus_words = [idx for doc_idxs in vidx_corpus for idx in doc_idxs]

In [ ]:
# holders for dimensions we will reference often
n_docs = len(vidx_corpus)
n_vocab = len(corpus_vocab)
n_words = len(corpus_words)
n_docs, n_vocab, n_words

In [ ]:
# set the number of topics into which the corpus will separate words and docs 
n_topics = 5

In [ ]:
# dirichlet priors (not updated in this implementation, only doing inference)
alpha = 1
gamma = 1

z_mn = [] # maps each word instance in each doc to a topic
p_z = np.ones(n_topics) * (1/n_topics) # unit vector from which to randomly sample initial z_mn

n_w_k = np.zeros((n_topics, n_vocab)) # count of each vocab word assigned to topic 
n_k = np.zeros(n_topics) # count of any word assigned to topic
n_m_k = np.zeros((n_topics, n_docs)) # count of each doc's words to topic

theta = np.zeros((n_docs, n_topics)) # document topic distribution
beta = np.zeros((n_vocab, n_topics)) # vocab word topic distribution 

In [ ]:
# sample p_z for each z_mn entry and get the initial counts
for m in range(n_docs):
    z_m = []
    for n in range(len(vidx_corpus[m])):
        k = np.random.multinomial(1, p_z).argmax()
        z_m.append(k)

        vidx = vidx_corpus[m][n]
        n_w_k[k, vidx] += 1
        n_k[k] += 1
        n_m_k[k, m] += 1
    
    z_mn.append(np.asarray(z_m))

In [ ]:
# update p_z for each z_mn and resample, then update the counts
def infer_z(n_docs, vidx_corpus, z_mn, n_w_k, n_k, n_m_k):
    n_changes = 0
    for m in range(n_docs):
        for n in range(len(vidx_corpus[m])):
            k = z_mn[m][n]

            vidx = vidx_corpus[m][n]
            n_w_k[k, vidx] -= 1
            n_k[k] -= 1
            n_m_k[k, m] -= 1

            numer = (n_w_k[:, vidx] + gamma) * (n_m_k[:, m] + alpha)
            denom = (n_k + n_vocab*gamma) * (len(vidx_corpus[m]) + n_topics*alpha)
            p_z = numer / denom
            p_z = p_z / p_z.sum()
            new_k = np.random.multinomial(1, p_z).argmax()

            z_mn[m][n] = new_k
            n_w_k[new_k, vidx] += 1
            n_k[new_k] += 1
            n_m_k[new_k, m] += 1

            if new_k != k:
                n_changes += 1

    return z_mn, n_w_k, n_k, n_m_k, n_changes

In [ ]:
# update theta using stats from z
def infer_theta(n_docs, n_m_k, alpha, vidx_corpus, n_topics, theta):
    for m in range(n_docs):
        numer = n_m_k[:, m] + alpha
        denom = len(vidx_corpus[m]) + n_topics*alpha
        theta[m] = numer / denom
            
    return theta

In [ ]:
# update beta using stats from z
def infer_beta(n_vocab, n_w_k, n_k, gamma, beta):
    for n in range(n_vocab):
        numer = n_w_k[:, n] + gamma
        denom = n_k + n_vocab*gamma
        beta[n] = numer / denom
    
    return beta      

In [ ]:
# compute the log likelihood of the corpus using theta and beta
def compute_logp_corpus(n_docs, alpha, vidx_corpus, z_mn, beta, theta):
    logp_corpus = Decimal(0)
    for m in range(n_docs):
        p_doc = Decimal(1)
        for n in range(len(vidx_corpus[m])):
            vidx = vidx_corpus[m][n]
            k = z_mn[m][n]
            p_doc *= Decimal(beta[vidx][k]) * Decimal(theta[m][k])
        
        logp_corpus += p_doc.ln()
    
    return logp_corpus

In [ ]:
# get the top words (by probability) associated with a given topic identified in the corpus
def get_topic_words(beta, k, n_top_words, corpus_vocab):
    topic_word_probs = beta[:, k]
    topic_word_indices = sorted(range(len(topic_word_probs)),
                                key=lambda i: topic_word_probs[i],
                                reverse=True)[:n_top_words]
    topic_words = [(corpus_vocab[i], beta[i, k]) for i in topic_word_indices]
        
    return topic_words   

In [ ]:
# functon to pretty print topic words and their probabilities
def print_topic_words(n_topics, beta, n_top_words, corpus_vocab, k)
    for k in range(n_topics):
        top_words = get_topic_words(beta, k, 10, corpus_vocab)
        words, probs = zip(*top_words)
        formatted_probs = ['{:.4f}'.format(prob) for prob in probs]

        print(f'Topic {k} top words:')
        pt = PrettyTable()
        pt.add_column('Words', words)
        pt.add_column('Probabilities', formatted_probs)
        print(pt)

In [ ]:
# simple plotting function
def plot_logp_corpus(logp_corpus_list):
    fig, ax = plt.subplots(constrained_layout=True)
    ax.plot(logp_corpus_list)
    ax.set_xlim(0, 256)
    ax.set_xlabel('Iterations')
    ax.set_ylabel('Corpus Log Likelihood')
    ax.set_yticklabels(['{:,}'.format(int(logp)) for logp in ax.get_yticks().tolist()])
    plt.savefig('logp_corpus_vs_iters.png')

In [ ]:
# perform inference and collect the log likelihood at each iteration
logp_corpus_list = []
for iters in range(2^8):
    z_mn, n_w_k, n_k, n_m_k, n_changes = infer_z(n_docs, vidx_corpus, z_mn, n_w_k, n_k, n_m_k)
    theta = infer_theta(n_docs, n_m_k, alpha, vidx_corpus, n_topics, theta)
    beta = infer_beta(n_vocab, n_w_k, n_k, gamma, beta)
    logp_corpus = compute_logp_corpus(n_docs, alpha, vidx_corpus, z_mn, beta, theta)
    logp_corpus_list.append(logp_corpus)
    
    print(f'\nFinished iteration {iters}...')
    print(f'Iteration z_mn topic change count: {n_changes}')
    print(f'Iteration logp_corpus: {logp_corpus:,}')